In [14]:
import tensorflow as tf
import numpy      as np
import pathlib
import data

from tensorflow.keras.optimizers  import Adam
from tensorflow.keras.losses      import CategoricalCrossentropy
from tensorflow.keras.layers      import Dense, Dropout, LSTM
from tensorflow.keras             import Sequential

In [15]:
def getSequences(datasetType):
    sequences = []
    labels    = []
    for dataRow in dataObj.data[:]:
        if(datasetType == dataRow[0]):
            sequencePath = pathlib.Path(r"/home/jupyter/action-recognition/Sequences")/dataRow[0]/dataRow[1]/(dataRow[2] + "_featureSequence.npy")
            sequence     = np.load(sequencePath)
            print(sequence)
            print(dataRow[2])
            sequences.append(sequence)
            label = dataObj.getClassIndex(dataRow[1])
            labels.append(label)
    return np.array(sequences), np.array(labels, dtype=np.uint8)

In [16]:
def getDataset(sequences, labels):
    dataset = tf.data.Dataset.from_tensor_slices((sequences, labels))
    return dataset

In [17]:
def prepareTrainDataset(dataset, cache, shuffleBufferSize):
    if cache:
        if isinstance(cache, str):
            dataset = dataset.cache(cache)
        else:
            dataset = dataset.cache()
    
    dataset = dataset.shuffle(buffer_size = shuffleBufferSize)
    # Repeat forever
    dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE)
    # `prefetch` lets the dataset fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset

In [18]:
def getModel():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(256, input_shape=(None, 2048)))
    model.add(tf.keras.layers.Dense(64, activation = 'relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(numClasses, activation = 'softmax')) # change 3 to numClasses
    
    model.compile(optimizer = tf.keras.optimizers.Adam(lr = 0.0001),
                  loss      = tf.keras.losses.CategoricalCrossentropy(from_logits = True),
                  metrics   = ['accuracy'])
    return model

In [19]:
def trainModel(model, epochs, trainDataset, validationDataset, steps_per_epoch, validation_steps, callbacks):
    history = model.fit(trainDataset, 
                        epochs = epochs,
                        validation_data  = validationDataset,
                        steps_per_epoch  = steps_per_epoch,
                        validation_steps = validation_steps,
                        callbacks = callbacks)
    return model, history

In [20]:
dataObj    = data.Data()
numClasses = dataObj.numClasses
AUTOTUNE   = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

In [21]:
def main():
    epochs = 10
    
    cacheFilePath = "./trainRNNDatasetCache"
    
    trainSequences, trainLabels           = getSequences('Train')
    validationSequences, validationLabels = getSequences('Validation')
    
    trainDataset      = getDataset(trainSequences, trainLabels)
    validationDataset = getDataset(validationSequences, validationLabels)
    
    trainDataset      = prepareTrainDataset(trainDataset, cacheFilePath, 100)
    validationDataset = validationDataset.batch(BATCH_SIZE)
    
    trainSeqCount      = dataObj.getDatasetCount('Train')
    validationSeqCount = dataObj.getDatasetCount('Validation')
    
    steps_per_epoch  = np.ceil(trainSeqCount/BATCH_SIZE)
    validation_steps = np.ceil(validationSeqCount/BATCH_SIZE)
    
    model = getModel()
    trained_model, history = trainModel(model, epochs, trainDataset, validationDataset, steps_per_epoch, validation_steps, [])

In [22]:
main()

[[0.11238838 0.08315331 0.04892578 ... 0.59028476 0.06388988 0.09357292]
 [0.08219181 0.07715625 0.07221513 ... 0.59583414 0.05957212 0.06265104]]
v_ApplyEyeMakeup_g11_c01
[[0.10666363 0.10650602 0.02439253 ... 0.5749303  0.17032726 0.07399839]
 [0.21393049 0.09799025 0.09160873 ... 0.3971149  0.1407288  0.09231264]]
v_ApplyEyeMakeup_g11_c02
[[0.03705182 0.05659282 0.00862073 ... 0.6080284  0.15420984 0.4691834 ]
 [0.05338989 0.074021   0.03035915 ... 0.5466183  0.124742   0.31635594]]
v_ApplyEyeMakeup_g11_c03
[[0.1630912  0.19442303 0.06964789 ... 0.4769309  0.11579423 0.22494629]
 [0.13687609 0.1395269  0.09368806 ... 0.54142296 0.17943707 0.2923126 ]]
v_ApplyEyeMakeup_g11_c04
[[0.13913058 0.03696126 0.05766505 ... 0.53251785 0.16325063 0.05361369]
 [0.13334122 0.03728019 0.04697704 ... 0.43987188 0.19024383 0.08113354]]
v_ApplyEyeMakeup_g11_c05
[[0.08276644 0.46725056 0.04631275 ... 0.37352225 0.15852657 0.2218684 ]
 [0.1229119  0.4685181  0.08335836 ... 0.30902573 0.14482614 0.2114